In [12]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 28.8 MB/s 


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.8.2


In [5]:
fashion_mnist = keras.datasets.fashion_mnist

In [6]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [7]:
train_images = train_images/255.0
test_images = test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:

train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model 

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [14]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [15]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 21s]
val_accuracy: 0.9053333401679993

Best val_accuracy So Far: 0.9108333587646484
Total elapsed time: 00h 02m 11s


In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 112)       1120      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        89632     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 112)               1734768   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 1,826,650
Trainable params: 1,826,650
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1431 - accuracy: 0.9452 - val_loss: 0.2409 - val_accuracy: 0.9187
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1016 - accuracy: 0.9618 - val_loss: 0.2680 - val_accuracy: 0.9152
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0729 - accuracy: 0.9734 - val_loss: 0.3188 - val_accuracy: 0.9163
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0522 - accuracy: 0.9808 - val_loss: 0.3650 - val_accuracy: 0.9108
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0396 - accuracy: 0.9857 - val_loss: 0.4238 - val_accuracy: 0.9125
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0296 - accuracy: 0.9895 - val_loss: 0.4808 - val_accuracy: 0.9120
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0272 - accuracy: 0.9901 - val_loss: 0.4928 - val_accuracy